In [1]:
import os
import librosa
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model

In [3]:
from transformers import Wav2Vec2Model

# Laden des Modells
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-xls-r-300m")

# Anzahl der Transformer-Schichten im Modell
num_layers = model.config.num_hidden_layers

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

import os

# Setzen Sie den Pfad zu dem Verzeichnis, das Ihre MP3-Dateien enthält
directory_path = '/content/drive/My Drive/NLP/clips__test'

print(len(os.listdir(directory_path)))

Mounted at /content/drive
20


In [5]:
os.listdir(directory_path)[:10]

['gut_acf67674-c912-42c0-ba3c-f85e2db965ac',
 'gut_4ee1811c-9884-4261-99cb-2f7346c8ea6e',
 'gut_684dd9cf-2844-407b-9a7f-12e7b559773f',
 'gut_005039b8-898f-48d8-b7cc-8a16bd1055c8',
 'gut_86c0e008-eebd-44ab-ac69-23706252a2e5',
 'gut_8b48025a-2cff-4346-9dab-b45cfb683e22',
 'gut_31cab952-98eb-45cb-a243-c951519c5c40',
 'gut_0d54807c-ec46-415f-8ef2-0a2f769aab1a',
 'gut_8050767b-0a0e-43db-8754-2a42e896f7dd',
 'gut_d9be733d-d0bd-41bb-8dc5-dfcabb99c105']

In [ ]:
import os
import librosa
import torch
from tqdm import tqdm
import numpy as np
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model

# Gerät festlegen, GPU wenn verfügbar, sonst CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-xls-r-300m")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-xls-r-300m", output_hidden_states=True)
model.to(device)  # Modell auf das festgelegte Gerät verschieben

def load_audio_files(directory, layer_indices=[-1]):
    """Lädt alle MP3-Dateien im angegebenen Verzeichnis und extrahiert die Repräsentationen aus den spezifizierten Schichten."""
    for filename in tqdm(os.listdir(directory)):
        if filename.endswith(".mp3"):
            file_path = os.path.join(directory, filename)
            audio, sr = librosa.load(file_path, sr=16000)
            input_values = feature_extractor(audio, return_tensors="pt", sampling_rate=sr).input_values
            input_values = input_values.to(device)  # Eingabedaten auf das Gerät verschieben
            with torch.no_grad():
                outputs = model(input_values)
                for index in layer_indices:
                    hidden_states = outputs.hidden_states[index]
                    # Erstellt einen Unterordner im selben Verzeichnis für jede Schicht, falls nicht vorhanden
                    layer_dir = os.path.join(directory, f"layer_{index}")
                    os.makedirs(layer_dir, exist_ok=True)
                    save_path = os.path.join(layer_dir, f"{os.path.splitext(filename)[0]}_layer_{index}.npy")
                    np.save(save_path, hidden_states.cpu().numpy())
                    # print(f"Verarbeitet und gespeichert: {filename} in {save_path}")



for d in os.listdir(directory_path):
  dir = os.path.join(directory_path, d)
  load_audio_files(dir, layer_indices=[0, 5, 10, 15, 20, 24])

Using device: cuda


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  1%|          | 3/362 [00:02<04:11,  1.43it/s]